# KITTI-360

This notebook lets you instantiate the **[KITTI-360](http://www.cvlibs.net/datasets/kitti-360)** dataset from scratch and visualize **3D+2D cylindrical samples**.

Note that you will need **at least 64G** available for the KITTI-360 raw dataset and **at least 250G** for the processed files at **5cm voxel resolution** and **1408x376 image resolution**. 

The KITTI-360 dataset is composed of **sequences** of same-day acquisition sessions in a large urban area. These are spatially divided into **windows** of roughly 3M points each. For training, we sample these windows with **3D cylinders and all images seeing at least one point in the cylinders**.

Such dataset cannot be loaded all at once into a **64G RAM** memory, so the `KITTI360Dataset` class from `torch_points3d.datasets.segmentation.multimodal.kitti360` contains a memory manager that loads only a limited number of windows at once. At training time, the `KITTI360Tracker` from `torch_points3d.metrics.kitti360_tracker` prepares the epoch's samples so as to minimize windows load time while ensuring the diversity of the batch items.

In [ ]:
# Select you GPU
I_GPU = 0

In [ ]:
# Uncomment to use autoreload
# %load_ext autoreload
# %autoreload 2

import os
import os.path as osp
import sys
import torch
import numpy as np
from time import time
from omegaconf import OmegaConf
start = time()
import warnings
warnings.filterwarnings('ignore')

torch.cuda.set_device(I_GPU)
DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)

from torch_points3d.utils.config import hydra_read
from torch_geometric.data import Data
from torch_points3d.core.multimodal.data import MMData
from torch_points3d.visualization.multimodal_data import visualize_mm_data
from torch_points3d.core.multimodal.image import SameSettingImageData, ImageData
from torch_points3d.datasets.segmentation.multimodal.kitti360 import KITTI360DatasetMM

If `visualize_mm_data` does not throw any error but the visualization does not appear, you may need to change your plotly renderer below.

In [ ]:
import plotly.io as pio

pio.renderers.default = 'jupyterlab'        # for local notebook
# pio.renderers.default = 'iframe_connected'  # for remote notebook. Other working (but seemingly slower) options are: 'sphinx_gallery' and 'iframe'

## Dataset creation

The following will instantiate the dataset. If the data is not found at `DATA_ROOT`, the folder structure will be created there and the raw dataset will be downloaded there. 

**Memory-friendly tip** : if you have already downloaded the dataset once and simply want to instantiate a new dataset with different preprocessing (*e.g* change 3D or 2D resolution, mapping parameterization, etc), I recommend you manually replicate the folder hierarchy of your already-existing dataset and create a symlink to its `raw/` directory to avoid downloading and storing (very) large files twice.

You will find the config file ruling the dataset creation at `conf/data/segmentation/multimodal/kitti360-sparse.yaml`. You may edit this file or create new configs inheriting from this one using Hydra and create the associated dataset by modifying `config_file` accordingly in the following cell.

In [ ]:
# Set your dataset root directory, where the data was/will be downloaded
DATA_ROOT = '/path/to/your/dataset/root/directory'
mini = False                                                          # set to True to only load and play with a small portion of the KITTI-360 dataset
train_is_trainval = False                                             # set to True if you want to the Train set to be Train+Val
sample_per_epoch = 12000                                              # number of cylinders sampled in the Train set. Corrects class imbalance. Set to 0 for regularly-sampled cylinders

config_file = 'segmentation/multimodal/kitti360-sparse'   
models_config = 'segmentation/multimodal/sparseconv3d'                # this does not really matter here, but is expected by hydra for config parsing
model_name = 'Res16UNet34-PointPyramid-early-cityscapes-interpolate'  # this does not really matter here, but is expected by hydra for config parsing

overrides = [
    'task=segmentation',
    f'data={config_file}',
    f'data.mini={mini}',
    f'models={models_config}',
    f'model_name={model_name}',
    f'data.dataroot={DATA_ROOT}',
    f'+train_is_trainval={train_is_trainval}',
    f'data.sample_per_epoch={sample_per_epoch}',
]

cfg = hydra_read(overrides)
# print(OmegaConf.to_yaml(cfg)

The dataset will now be created based on the parsed configuration. I recommend having **at least 64G** available for the KITTI-360 raw dataset and **at least 250G** for the processed files at **5cm voxel resolution** and **1408x376 image resolution**. 

As long as you do not change core dataset parameters, preprocessing should only be performed once for your dataset. It may take some time, **mostly depending on the 3D and 2D resolutions** you choose to work with (the larger the slower). As a rule of thumb, it took me **about 1 minute** (excluding download) to preprocess the 2D and 3D data **for each of the 342 windows** that make up the KITTI-360 dataset.

In [ ]:
# Dataset instantiation
start = time()
dataset = KITTI360DatasetMM(cfg.data)
# print(dataset)
print(f"Time = {time() - start:0.1f} sec.")

To visualize the multimodal samples produced by the dataset, we need to remove some of the dataset transforms that affect points, images and mappings.

At training and evaluation time, these transforms are used for data augmentation, dynamic size batching (see our [paper](https://arxiv.org/submit/4264152)), etc...

In [ ]:
from torch_points3d.core.multimodal import MMData, ImageData
from torch_points3d.datasets.segmentation.kitti360 import KITTI360_NUM_CLASSES, INV_OBJECT_LABEL, OBJECT_COLOR, CLASS_NAMES, CLASS_COLORS

train_2d_transforms = dataset.train_dataset.transform_image.transforms
val_2d_transforms = dataset.val_dataset.transform_image.transforms
test_2d_transforms = dataset.test_dataset[0].transform_image.transforms

In [ ]:
dataset.train_dataset.transform = None  # removes 3D sample centering, rotation, jittering, etc...
dataset.train_dataset.transform_image.transforms = train_2d_transforms[:4]  # removes some image transforms
dataset.train_dataset.transform_image.transforms[3].credit = 1408 * 376 * 3  # pixel memory credit controlling how many images can be loaded in a sample

dataset.val_dataset.transform = None  # removes 3D sample centering, rotation, jittering, etc...
dataset.val_dataset.transform_image.transforms = val_2d_transforms[:4]  # removes some image transforms
dataset.val_dataset.transform_image.transforms[3].credit = 1408 * 376 * 3  # pixel memory credit controlling how many images can be loaded in a sample

dataset.test_dataset[0].transform = dataset.val_dataset.transform
dataset.test_dataset[0].transform_image = dataset.val_dataset.transform_image

## Visualize a single multimodal sample

We can now pick samples from the train, val and test datasets.

To allow rapid visualization, note the sample is subsampled to `voxel=0.5`. 

In [ ]:
# Pick a multimodal cylindrical sample
mm_data = dataset.train_dataset[dataset.train_dataset._pick_random_label_and_window()]  # pick a random cylinder in the Train set, with balanced class probabilites
# mm_data = dataset.val_dataset[np.random.randint(len(dataset.val_dataset[0]))]           # pick a random cylinder in the Val set
# mm_data = dataset.test_dataset[np.random.randint(len(dataset.val_dataset[0]))]          # pick a random cylinder in the Test set

visualize_mm_data(mm_data, figsize=1000, pointsize=3, voxel=0.5, show_2d=True, front='y', class_names=CLASS_NAMES, class_colors=CLASS_COLORS, alpha=2)

## Visualize a whole KITTI-360 window

We can also view a whole window at once.

To allow rapid visualization, note the window is subsampled to `voxel=1` and the maximum number of points shown is set `max_points=500000`. Besides `show_2d=False` prevents images from being displayed, which would overload the viewer.

In [ ]:
from torch_points3d.datasets.segmentation.kitti360_config import CLASS_NAMES, CLASS_COLORS
from torch_points3d.core.multimodal.data import MMData
from torch_points3d.core.multimodal.image import ImageData

# Index of the window
i_window = 0

# Load the window in memory
# dataset.train_dataset[i_window]                       # uncomment to get a Train window
# mm_window = dataset.train_dataset.buffer[i_window]    # uncomment to get a Train window
dataset.val_dataset[i_window]
mm_window = dataset.val_dataset.buffer[i_window]
# dataset.test_dataset[0][i_window]                     # uncomment to get a Test window
# mm_window = dataset.test_dataset[0].buffer[i_window]  # uncomment to get a Test window

mm_data_large = MMData(mm_window.data, image=ImageData(mm_window.images))

visualize_mm_data(mm_data_large, figsize=1000, pointsize=3, voxel=1, show_2d=False, front='map', class_names=CLASS_NAMES, class_colors=CLASS_COLORS, alpha=2, max_points=500000)